# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_df = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,margate,100,GB,1616043918,84,51.3813,1.3862,43.0,4.00
1,1,port elizabeth,75,ZA,1616043913,94,-33.9180,25.5701,69.8,5.75
2,2,ushuaia,75,AR,1616043919,66,-54.8000,-68.3000,53.6,5.75
3,3,qaanaaq,8,GL,1616043919,99,77.4840,-69.3632,-18.0,1.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

In [11]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
weights = weather_df["Humidity"].astype(float)

max_intensity = weather_df["Humidity"].max()

In [18]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=max_intensity, 
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
hotel_df = weather_df.loc[(weather_df["Cloudiness"] < 50) & 
                              (weather_df["Humidity"] < 100) & 
                              (weather_df["Wind Speed"] < 5) , :].copy()

hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,qaanaaq,8,GL,1616043919,99,77.484,-69.3632,-18.0,1.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df["Hotel Name"] = ""

In [33]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,3,qaanaaq,8,GL,1616043919,99,77.484,-69.3632,-18.0,1.41,


In [34]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "77.484,-69.3632",  # philadelphia coords,
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add keyword to params dict
#    params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

    

Retrieving Results for Index 3.
Closest hotel is Qaanaaq Airport Parking.
------------


In [35]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,3,qaanaaq,8,GL,1616043919,99,77.484,-69.3632,-18.0,1.41,Qaanaaq Airport Parking


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))